In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import time

In [2]:
# Premier League 2024-2025 Season 
url = "https://fbref.com/en/squads/822bd0ba/2024-2025/matchlogs/c9/schedule/Liverpool-Scores-and-Fixtures-Premier-League"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
response

<Response [200]>

In [12]:
# Matches already played in the PL 24-25 Season
match_report_tags = soup.find_all('td', {'data-stat': 'match_report'})
match_report_urls = []

for i in match_report_tags:
    if i.text == 'Match Report':
        match_report_urls.append("https://fbref.com/" + i.find('a')['href'])

df_list = []
for match_report_url in match_report_urls:
    df_list.append(extract_shooting_stats(match_report_url))

https://fbref.com//en/matches/a1d0d529/Ipswich-Town-Liverpool-August-17-2024-Premier-League
https://fbref.com//en/matches/09b1742e/Liverpool-Brentford-August-25-2024-Premier-League
https://fbref.com//en/matches/a7ab7a12/North-West-Derby-Manchester-United-Liverpool-September-1-2024-Premier-League
https://fbref.com//en/matches/674bfe9e/Liverpool-Nottingham-Forest-September-14-2024-Premier-League
https://fbref.com//en/matches/32a9539b/Liverpool-Bournemouth-September-21-2024-Premier-League
https://fbref.com//en/matches/f2633f1d/Wolverhampton-Wanderers-Liverpool-September-28-2024-Premier-League
https://fbref.com//en/matches/49ea224b/Crystal-Palace-Liverpool-October-5-2024-Premier-League
https://fbref.com//en/matches/99b4737c/Liverpool-Chelsea-October-20-2024-Premier-League
https://fbref.com//en/matches/68aa1099/Arsenal-Liverpool-October-27-2024-Premier-League
https://fbref.com//en/matches/7d114c70/Liverpool-Brighton-and-Hove-Albion-November-2-2024-Premier-League
https://fbref.com//en/matche

In [15]:
pd.concat(df_list).to_csv('data/df.csv',index=False)

In [17]:
temp = pd.concat(df_list)

In [23]:
temp[temp.outcome == 'Goal']

,match,player,xG,PSxG,outcome,distance,body_part
1,Ipswich Town vs. Liverpool Match Report – Satu...,Mohamed Salah,0.41,0.82,Goal,7,Left Foot
2,Liverpool vs. Brentford Match Report – Sunday ...,Mohamed Salah,0.43,0.58,Goal,12,Left Foot
0,Manchester United vs. Liverpool Match Report –...,Mohamed Salah,0.20,0.77,Goal,16,Left Foot
2,Arsenal vs. Liverpool Match Report – Sunday Oc...,Mohamed Salah,0.29,0.79,Goal,13,Left Foot
2,Liverpool vs. Brighton & Hove Albion Match Rep...,Mohamed Salah,0.06,0.16,Goal,16,Left Foot
2,Liverpool vs. Aston Villa Match Report – Satur...,Mohamed Salah,0.67,0.72,Goal,6,Left Foot
3,Southampton vs. Liverpool Match Report – Sunda...,Mohamed Salah,0.52,0.88,Goal,17,Left Foot
1,Newcastle United vs. Liverpool Match Report – ...,Mohamed Salah,0.44,0.96,Goal,5,Right Foot
3,Newcastle United vs. Liverpool Match Report – ...,Mohamed Salah,0.13,0.86,Goal,11,Left Foot
5,Tottenham Hotspur vs. Liverpool Match Report –...,Mohamed Salah,0.87,1.00,Goal,4,Left Foot


In [28]:
def extract_shooting_stats(match_report_url):
    print(match_report_url)
    time.sleep(10)  # Adds a 2-second delay before making a request
    
    response = requests.get(match_report_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    match_title = soup.find('h1').text
    shots_table = soup.find('table', {'id': 'shots_all'})
    
    match_list = []
    player_list = []
    xg_list = []
    psxg_list = []
    outcome_list = []
    distance_list = []
    body_part_list = []
    
    for i in shots_table.find_all('tr'):
        player_element = i.find('td', {'data-stat': 'player'})

        
        if player_element:
            if player_element.text == 'Mohamed Salah' or player_element.text == 'Mohamed Salah (pen)':
                match_list.append(match_title)
                player_list.append(player_element.text)
                xg_list.append(i.find('td', {'data-stat':'xg_shot'}).text)
                psxg_list.append(i.find('td', {'data-stat':'psxg_shot'}).text)
                outcome_list.append(i.find('td', {'data-stat':'outcome'}).text)
                distance_list.append(i.find('td', {'data-stat':'distance'}).text)
                body_part_list.append(i.find('td', {'data-stat':'body_part'}).text)
    
    data = {'match': match_list, 'player': player_list, 'xG': xg_list, 'PSxG': psxg_list, 'outcome': outcome_list, 'distance': distance_list, 'body_part': body_part_list}
    
    df = pd.DataFrame(data)

    return df

In [29]:
print(extract_shooting_stats("https://fbref.com//en/matches/f2633f1d/Wolverhampton-Wanderers-Liverpool-September-28-2024-Premier-League"))

https://fbref.com//en/matches/f2633f1d/Wolverhampton-Wanderers-Liverpool-September-28-2024-Premier-League
Jean-Ricner Bellegarde
Jørgen Strand Larsen
Matheus Cunha
Luis Díaz
Trent Alexander-Arnold
Dominik Szoboszlai
Trent Alexander-Arnold
Ibrahima Konaté
Mario Lemina

Mohamed Salah
Rayan Aït-Nouri
Mohamed Salah (pen)
Mohamed Salah
Matheus Cunha
Carlos Forbs
Nélson Semedo
Curtis Jones
Ibrahima Konaté
                                               match               player  \
0  Wolverhampton Wanderers vs. Liverpool Match Re...        Mohamed Salah   
1  Wolverhampton Wanderers vs. Liverpool Match Re...  Mohamed Salah (pen)   
2  Wolverhampton Wanderers vs. Liverpool Match Re...        Mohamed Salah   

     xG  PSxG     outcome distance   body_part  
0  0.49  0.00  Off Target       17  Right Foot  
1  0.79  0.60        Goal       13   Left Foot  
2  0.04  0.05       Saved       19   Left Foot  
